In [ ]:
import numpy as np
import torch
import torch.nn as nn
import h5py
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
'''loading data'''
f = h5py.File("../modulation-prediction/data.hdf5", 'r')
X_train = torch.from_numpy(np.array(f['train']))
X_test = torch.from_numpy(np.array(f['test']))
Y_train = pd.read_csv('../modulation-prediction/train_labels.csv').to_numpy()

In [ ]:
class_index = {
 'FM': 0,
 'OQPSK':1,
 'BPSK':2,
 '8PSK':3,
 'AM-SSB-SC':4,
 '4ASK':5,
 '16PSK':6,
 'AM-DSB-SC':7, 
 'QPSK': 8, 
 'OOK':9
}

classnum = 10
def classToIndex(catg):
    return class_index[catg]

# turn a class into a <1 x 10> Tensor
def classToTensor(catg):
    tensor = torch.zeros(1, classnum)
    tensor[0][classToIndex(catg)] = 1
    return tensor

# Turn dataset into a <lines x 1 x classnum>,
# or an array of one-hot letter vectors
def setToTensor(dataset):
    tensor = torch.zeros(dataset.shape[0], 1, classnum)
    for li, catg in enumerate(dataset[:,1]):
        tensor[li][0][classToIndex(catg)] = 1
    return tensor

#direct mapping, not one-hot
def setToNum(dataset):
    tensor = torch.zeros(dataset.shape[0])
    for li, catg in enumerate(dataset[:,1]):
        tensor[li] = classToIndex(catg)
    return tensor

In [ ]:
# Hyper-parameters
sequence_length = 1024
input_size = 2
hidden_size = 4
num_layers = 1
num_classes = 10
batch_size = 32
num_epochs = 2
learning_rate = 0.01

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
total_step = X_train.shape[0]
for epoch in range(num_epochs):
        modulation = X_train
        labels = setToNum(Y_train)
#         print(labels.shape)
        
        # Forward pass
        outputs = model(modulation)
        print(outputs.shape)
#         _, trainpredicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 30000
    
    trainoutput = model(X_train)
    _, trainpredicted = torch.max(trainoutputs.data, 1)
    traincorrect += (trainpredicted == labels).sum().item()
    
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    prediction = pd.DataFrame(predicted, header = True, index = False)
    data1.to_csv('../output/test_prediction.csv')
        
    print('Train Accuracy of the model on the 30000 train modulations: {} %'.format(100 * traincorrect / total)) 